In [1]:
# default_exp dcae

In [2]:
# hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

# DCAE: Deep Convolutional Autoencoder

> This notebook tries to apply the ideas of the paper [TimeCluster](https://link.springer.com/article/10.1007/s00371-019-01673-y),
with regard to the application of DCAEs for compressing multivariate time series data.

In [3]:
# export
from fastcore import test
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool1D, Reshape, UpSampling1D, InputLayer

In [4]:
#hide
from pacmel_mining_use_case.load import *
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbCallback
from yaml import load, FullLoader
from fastcore.utils import Path
from datetime import datetime
import pickle

For the experiment tracking and hyperparameter we will use the tool **Weights & Biases**. The actual version of the library that we are using can be installed via `
pip install https://github.com/wandb/client/archive/artifacts/next.zip`. It contains features that are not part of the public release version, such as the management of artifacts for things like dataset versioning.

Before running this notebook, make sure you run in a terminal `wandb login [API_KEY]`. You can see your API_KEY in the settings of your wandb account.

In [5]:
# hide
run_dcae = wandb.init(project="timecluster-extension",
                      job_type='train_DCAE',
                      allow_val_change=True,
                      resume=False)
config = wandb.config  # Object for storing hyperparameters

wandb: Wandb version 0.9.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Loading the dataset

To load the dataset we will download a specific dataset artifact from the collection of artifacts
stored in the weights and biases (wandb) project associated to this experiment.

In [6]:
artifact_name_and_version = 'JNK:v6'

In [7]:
ds_artifact = run_dcae.use_artifact(artifact=artifact_name_and_version, type='dataset')

wandb: Wandb version 0.9.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [8]:
# parameters (uncomment to override the yaml file)
config.update(allow_val_change=True,
              params={
                  'ds_artifact_type': ds_artifact.type,
                  'ds_artifact_name': ds_artifact.name,
                  'ds_artifact_digest': ds_artifact.digest
              })
ds_artifact.type, ds_artifact.name, ds_artifact.digest

wandb: Wandb version 0.9.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


('dataset', 'JNK:v6', '423ff6f4642accdded6953b60ab63974')

The artifact must have been logged from a `TSArtifact` object. IF that's true, the metadata of the downloaded artifact will contain all the necessary information to recover the dataframe containing the time series.

In [9]:
ds_artifact.metadata

{'TS': {'ed': '2019-06-30 00:00:00',
  'sd': '2019-06-01 00:00:00',
  'freq': '<5 * Seconds>',
  'hash': '-2215838723830694101',
  'stds': {'SM_TotalRoute': 0.9984699786811964,
   'SM_ShearerSpeed': 0.9914461896465906,
   'LHD_EngineCurrent': 0.980337055406906,
   'RHD_EngineCurrent': 0.9802546618257328,
   'SM_ShearerLocation': 0.9999981418126948,
   'SM_ShearerMoveInLeft': 0.9848440588911728,
   'LA_LeftArmTemperature': 0.9992066555701188,
   'SM_ShearerMoveInRight': 0.9851666027617088,
   'RA_RightArmTemperature': 0.9994430004503356,
   'RCD_BearingTemperature': 0.9993778850397812,
   'SM_DailyRouteOfTheShearer': 0.9998100911253166,
   'LP_AverageThree-phaseCurrent': 0.9367778108343784,
   'LCD_AverageThree-phaseCurrent': 0.9579865673025332,
   'RCD_AverageThree-phaseCurrent': 0.95666130033227,
   'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)': 0.9994649828913424,
   'RHD_RightHaulageDrive(tractor)Temperature(gearbox)': 0.9997118247614564},
  'vars': ['RCD_AverageThree-phaseCur

In [10]:
df = ds_artifact.to_df()

wandb: Downloading large artifact JNK:v6, 65.00MB. 1 files... Done. 0.2s


The data that will be used in the rest of the notebook will be stored in the dataframe `df`

In [11]:
df

,RCD_AverageThree-phaseCurrent,LCD_AverageThree-phaseCurrent,LP_AverageThree-phaseCurrent,LHD_LeftHaulageDrive(tractor)Temperature(gearbox),RHD_RightHaulageDrive(tractor)Temperature(gearbox),LA_LeftArmTemperature,RA_RightArmTemperature,SM_DailyRouteOfTheShearer,SM_TotalRoute,LHD_EngineCurrent,RHD_EngineCurrent,RCD_BearingTemperature,SM_ShearerSpeed,SM_ShearerLocation,SM_ShearerMoveInLeft,SM_ShearerMoveInRight
TIMESTAMP,,,,,,,,,,,,,,,,
2019-06-01 00:00:00,-0.929460,-0.918607,-0.669569,-2.504799,-0.229084,-3.599404,-3.372501,-1.065108,-6.186766,-0.58024,-0.578736,-3.526231,-0.388547,0.807398,-0.280183,-0.34113
2019-06-01 00:00:05,-0.929460,-0.918607,-0.669569,-2.504799,-0.229084,-3.599404,-3.372501,-1.065108,-6.186766,-0.58024,-0.578736,-3.526231,-0.388547,0.807398,-0.280183,-0.34113
2019-06-01 00:00:10,-0.929460,-0.918607,-0.669569,-2.504799,-0.229084,-3.599404,-3.372501,-1.065108,-6.186766,-0.58024,-0.578736,-3.526231,-0.388547,0.807398,-0.280183,-0.34113
2019-06-01 00:00:15,-0.929460,-0.918607,-0.669569,-2.504799,-0.229084,-3.599404,-3.372501,-1.065108,-6.186766,-0.58024,-0.578736,-3.526231,-0.388547,0.807398,-0.280183,-0.34113
2019-06-01 00:00:20,-0.929460,-0.918607,-0.669569,-2.504799,-0.229084,-3.599404,-3.372501,-1.065108,-6.186766,-0.58024,-0.578736,-3.526231,-0.388547,0.807398,-0.280183,-0.34113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-29 23:59:40,-0.612055,-0.641547,-0.669569,0.787406,-0.229084,0.415665,0.293562,-0.333862,1.472100,-0.58024,-0.578736,0.253170,-0.388547,0.781844,-0.280183,-0.34113
2019-06-29 23:59:45,-0.612055,-0.641547,-0.669569,0.787406,-0.229084,0.415665,0.293562,-0.333862,1.472100,-0.58024,-0.578736,0.253170,-0.388547,0.781844,-0.280183,-0.34113
2019-06-29 23:59:50,-0.612055,-0.641547,-0.669569,0.787406,-0.229084,0.415665,0.293562,-0.333862,1.472100,-0.58024,-0.578736,0.253170,-0.388547,0.781844,-0.280183,-0.34113


## Train

### Sliding window features

Define a continuous multivariate time-series data $D$ of dimension $d$ with $n$ time-steps, $D = X_1,X_2,\dots,X_n$ , where each $X_i = \{x_i^1,\dots,x_i^d\}$ . Let $w$ be the window width, $s$ the stride, and $t$ the start time of a sliding window in the data.

Define a new matrix $Z_k$ where each row is a vector of size $w$ of data extracted from the $k^{th}$ dimension.

\begin{aligned}&Z_k(w,s,t)\\&\quad =\begin{bmatrix} x_{t}^k&\quad x_{t+1}^k&\quad \dots&\quad x_{t+w-1}^k \\ x_{t+s}^k&\quad x_{t+s+1}^k&\quad \dots&\quad x_{t+s+w-1}^k \\ \vdots&\quad \vdots&\quad \ddots&\quad \vdots \\ x_{t+(r-1)s}^k&\quad x_{t+(r-1)s+1}^k&\quad \dots&\quad x_{t+(r-1)s+w-1}^k \end{bmatrix} \end{aligned}

where $r$ is the number of desired rows, and $t+(r-1)s+w-1 \le n$


$Z$ is a $w \times s \times t$ matrix. The first step consists in slicing the original multivariate time series into slices of shape ($w \times d$), as shown in this figure from the paper.
<img src="https://i.imgur.com/R9Fx8uO.png" style="width:800px;height:400px"/>

In [13]:
# parameters (uncomment to override the yaml file)
config.update(allow_val_change=True,
              params={
                  'w': 48,
                  'stride': 1,
                  't': 0  # Not supported yet
              })

wandb: Wandb version 0.9.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
test.equals(config.w % 12, 0)

True

The sliced data must be converted to a numpy array with shape $(n \times w \times d)$, where $n$ is the length of the time series, $w$ is the window size and $d$ is the number of dimensions in the time series. 

In [ ]:
# export utils
def df_slicer(df, w, s=1, padding=False, padding_value=0, return_as='ndarray'):
    "Transform a numeric dataframe `df` into slices (np arrays) of `w` \
    rows and the same number of columns than the original dataframe. The \
    distance between each slice is given by the stride `s`. If `padding` is \
    equals to True, the last slices which have less than `w` points are filled \
    with the value marked in the argument `padding_value`. Otherwise, those \
    slices are removed from the result."
    aux = [df.iloc[x:x+w] for x in range(0, len(df), s)]
    if padding:
        with_padding = [x.append(pd.DataFrame(
            np.full((w - len(x), len(df.columns)), padding_value),
            columns=df.columns.values)) if len(x) < w else x for x in aux]
    else:
        with_padding = [x for x in aux if len(x) == w]
    return np.rollaxis(np.dstack([x.values for x in with_padding]), -1)

In [ ]:
df_slices = df_slicer(df, w=config.w, s=config.stride)
len(df_slices)

In [ ]:
df_slices[0][0]

Test the number of slices and the size of each slice

In [ ]:
expected_nwindows = (int)((len(df) - config.w)/config.stride + 1)
expected = [(config.w, len(df.columns))]*expected_nwindows
actual = [x.shape for x in df_slices]
test.all_equal(expected, actual)

In [ ]:
# export load
def slices2array(slices):
    "`slices` is a list of dataframes, each of them containing an slice of a multivariate time series."
    return np.rollaxis(np.dstack([x.values for x in slices]), -1)

In [ ]:
input_data = slices2array([x for x in df_slices])

In [ ]:
test.equals(input_data.shape, (len(df_slices), len(df_slices[0]), len(df_slices[0].columns)))

At this point, it is very handy to provide a function to preprocess a dataframe as a sliced tensor suitable for model.

In [ ]:
# export load
def fmultiTSloader(df, w, stride, **kwargs):
    "Preprocess a dataframe with multivariate time series from a df `df` or from set of paths `paths`, \
    preprocess it calling `fpreprocess_numeric_vars` \
    slice it into time windows of length `w` and stride `stride` calling `fslicer`, and \
    conert the result into a numpy array, suitable for ML libraries. Optional arguments for \
    the intermediate functions can be passed through `**kwargs`"
    df_slices = fslicer(df, w, stride)
    array_slices = slices2array([x for x in df_slices])
    return (df, df_slices, array_slices)

In [ ]:
f_df, f_df_slices, f_array_slices = fmultiTSloader(df, config.w, config.stride)

In [ ]:
test.equals(df, f_df)

In [ ]:
test.equals(df_slices, f_df_slices)

In [ ]:
test.equals(input_data, f_array_slices)

## Extract important features from the multivariate time series data through Deep Convolutional Autoencoders


Deep Convolutional Auto Encoders (DCAE) is a powerful method for learning high-level and mid-level abstractions from low-level raw data. It has the ability to extract features from complex and large time-series in an unsupervised manner. This is useful to overcome the complexity of multivariate time-series.

Compared to the conventional auto-encoder, DCAE has fewer parameters than the conventional auto-encoder which means less training time. Also, DCAE uses local information to reconstruct the signal while conventional auto-encoders utilize fully connected layers to globally do the reconstruction. DCAE is an unsupervised model for representation learning which maps inputs into a new representation space. It has two main parts which are the encoding part that is used to project the data into a set of feature spaces and the decoding part that reconstructs the original data. The latent space representation is the space where the data lie in the bottleneck layers.

The loss function of the DCAE is defined as the error between the input and the output. DCAE aims to find a code for each input by minimizing the mean squared error (MSE) between its input (original data) and output (reconstructed data). The MSE is used which assists to minimize the loss; thus, the network is forced to learn a low-dimensional representation of the input.

We will implement the DCAE of the paper [TimeCluster](https://link.springer.com/article/10.1007/s00371-019-01673-y), whose architecture is shown in the table below:

![](https://i.imgur.com/3EjuAfQ.png)

Note that, in the paper, the input shape is $60 \times 3$, due to multivariate time series has 3 variables and the window size is 60. Generally, the size of the input/output of the autoencoder will depend on the shape of each slice obtained in the previos step. The number of latent features to be discovered is $60$ in the table above, but we can consider this as a free hyperparameter $\delta$. Also, according to the paper: "*The number of feature maps, size of filter and depth of the model are set based on the reconstruction error on validation set.*". Thus, we must provide flexibility in the creation of the DCAE in terms of these hyperparameters.º

In case you are not using a config file, you can also uncomment the following cell and define the hyperparameters in the fly

In [ ]:
# hide
config.update(allow_val_change=True,
    params = {
        'lr': 0.0009044187712482472,
        'n_filters': [32, 16, 12],
        'filter_sizes': [20, 10, 10],
        'output_filter_size': 20,
        'pool_sizes': [2, 2, 3],
        'delta': config.w,
        'batch_size': 92,
        'epochs': 10,
        'val_pct': 0.2
    }
)

In [ ]:
# hide
test.all_equal([len(x) for x in [config.n_filters, config.filter_sizes, config.pool_sizes]], np.repeat(len(config.n_filters), 3))

### Create the model

The implementation of the DCAE is done using Keras.

In [ ]:
# export
def createDCAE(w, d, delta, n_filters=[64,32,12], filter_sizes=[10,5,5], pool_sizes=[2,2,3], output_filter_size=10):
    "Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions, \
    sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be \
    contained in the Dense layer of the network. The the number of features \
    maps (filters), the filter size and the pool size can also be adjusted."
    # Test that the parameterization of the model is correct
    # 1. n_filters, filter_sizes and pool_sizes have the same length
    assert test.all_equal([len(x) for x in [n_filters, filter_sizes, pool_sizes]], np.repeat(len(n_filters), 3))
    # 2. Test that the number of filters in the last convLayer is equal to the product of the pool sizes
    assert np.prod(pool_sizes) == n_filters[-1]
    # 3. Test that the product of pool sizes is a divisor of the window size
    assert w % np.prod(pool_sizes).all() == 0
    # Create the model
    model = Sequential()
    model.add(InputLayer(input_shape=(w,d)))
    for (i, x) in enumerate(n_filters):
        model.add(Conv1D(filters=n_filters[i], kernel_size=filter_sizes[i], activation='relu', padding='same'))
        model.add(MaxPool1D(pool_size=pool_sizes[i]))
    aux_shape = model.output_shape[1:]
    model.add(Flatten())
    model.add(Dense(units=np.prod(aux_shape), activation='linear', name='latent_features'))
    model.add(Reshape(target_shape=aux_shape))
    for i, x in reversed(list(enumerate(n_filters))):
        model.add(Conv1D(filters=n_filters[i], kernel_size=filter_sizes[i], activation='relu', padding='same'))
        model.add(UpSampling1D(size=pool_sizes[i]))
    model.add(Conv1D(filters=d, kernel_size=output_filter_size, activation='linear', padding='same'))
    return model

In [ ]:
m = createDCAE(config.w, input_data.shape[-1], config.delta, n_filters=config.n_filters, 
              filter_sizes=config.filter_sizes, pool_sizes=config.pool_sizes,
              output_filter_size=config.output_filter_size)
m.summary()

In [ ]:
opt = Adam(learning_rate=config.lr)

In [ ]:
m.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_squared_error'])

In [ ]:
history = m.fit(x=input_data, y=input_data, batch_size=config.batch_size, 
      validation_split=config.val_pct, epochs=config.epochs, verbose=0, 
      callbacks=[WandbCallback(log_weights=True)])




To track the performance of this model fit, go to the project dashboard in Weights & Biases. The link is provided at the beginning of this notebook, after the execution of the function `wandb.init()'' 